# 日本人_Kaggle 初心者のnotebook Part1
私自身,初心者のため,まずどの様なデータがあり,どの様な処理をするべきなのかを見て  
どの様な既存モデルのがあるかを調べていきたいと思います。

# 0. 自分でできることがあるか考えてみた

このコンペでは,あるcommentの有害度合いを評価していきます.  
そして,ランキングの一致度合いでモデル評価されます.
そのため,予測値自体は自由に自分で設定できます.それと同時に設定する必要があります.  
また,教師データとしてless か more の二つを比べているだけのため,どのように教師あり学習(教師なし?)を回すべきか現状見通しが立ちません.  

ただ,二値分類問題として考えて作成すれば良さそうです.

そもそもとして,与えられたデータはvalidとあるためmodelを他の大規模データセットで学習させて,それの指標を見るためだけにvalidのdataを使うべきなのかもしれません.

そのため,fine-tuningなども出来ないのかもしれません.

# 1. Dataを見てみる
```
from pandas_profiling import ProfileReport 
```
が便利なので,それを使ってみます.
(今回は使う必要ない気もします.)

In [ ]:
import time

In [ ]:
%%time
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from pandas_profiling import ProfileReport

In [ ]:
%%time
valid_path="/kaggle/input/jigsaw-toxic-severity-rating/validation_data.csv"
comment_path="/kaggle/input/jigsaw-toxic-severity-rating/comments_to_score.csv"

In [ ]:
!pwd

In [ ]:
%%time
valid_df=pd.read_csv(valid_path)
comment_df=pd.read_csv(comment_path)

In [ ]:
%%time
tmp=pd.read_csv("../input/jigsaw-toxic-severity-rating/validation_data.csv")

In [ ]:
tmp.head()

In [ ]:
comment_df.head()

少し疑問ですが,今いるpathと関係なくimportできているの不思議ですね.  
```
../input/*
```
とした方が良いのかと思いました...(もちろんこっちでもできます)  
os.walkがおそらく機能していると思うのですが,本筋でないのであとでlocalで実験をし確認します.

In [ ]:
%%time
report = ProfileReport(valid_df)
report.to_file(os.path.join("", "train_report.html"))

In [ ]:
report = ProfileReport(comment_df)
report.to_file(os.path.join("", "comment_report.html"))

reportを見る限り,全てdistinctでworkerは同じものがいくつかあります.  
workerは飽くまで誰がその判断をしたかの,idであり,どのひとがどの指標で判断しているかといったことはわかるかもしれないですが, これをどの様にモデルに組み込むべきかも少し難しいと感じました.(そもそもvalidのため使えないかもしれないが,このvalidにoverfitする形ではダメな可能性があると再認識したい)  

one-hot-vectorとして組み込んでも良いですが,人の数だけ要素を増やすこととなり安定したモデルを作れません.  つまり,その人が真の判断者に近いかどうかの連続値を持たせられれば良いかもしれません.

ここで,Dataで紹介されている過去のコンペティションの内容とHotなDiscussionとCodeの内容を追ってみることにします.

# 1. 過去のコンペティションの内容を追ってみた

詳細のcodeは追って追う様にする.

## a. [Toxic Comment Classification Challenge](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge)

複数の指標において,true_positive_rateとfalse_positive_rateからなるROC曲線のAUCの平均値でモデルを作成する. -> これの優秀なモデルは使えそう  

これの指標だとmax_scoreは1.0のはずなのに,なぜかoverしているcodeがあった.

[Classifying multi-label comments (0.9741 lb)](https://www.kaggle.com/rhodiumbeng/classifying-multi-label-comments-0-9741-lb)

## b. [Jigsaw Unintended Bias in Toxicity Classification](https://www.kaggle.com/c/jigsaw-unintended-bias-in-toxicity-classification)



このコンペでは独自のmetricでモデルを評価する.基本的にAUCと同じだが,色々なケースにおいてのAUCを考えているっぽい(詳細は論文([Borkan, Daniel, et al. "Nuanced metrics for measuring unintended bias with real data for text classification." Companion proceedings of the 2019 world wide web conference. 2019.](https://arxiv.org/abs/1903.04561))を読む必要がある).

これも先ほどと同様に使えそう.

[Wombat Inference Kernel](https://www.kaggle.com/iezepov/wombat-inference-kernel)

## c. [Jigsaw Multilingual Toxic Comment Classification](https://www.kaggle.com/c/jigsaw-multilingual-toxic-comment-classification)

今回は一つの指標のROCで決まるmetricです.

[JMTC-20 lb.9508 Mono-lingual models](https://www.kaggle.com/mint101/jmtc-20-lb-9508-mono-lingual-models)

# 2. Hot なCodeとDiscussion

まだ,コンペが開始されたばかりなので,とりあえずdiscussionは無しにcodeをみることにします.

[AutoNLP for toxic ratings ;)](https://www.kaggle.com/abhishek/autonlp-for-toxic-ratings) では,
[autonlp_toxic_1](https://www.kaggle.com/abhishek/autonlp-toxic-1)というモデルを使用しています.

pytorchのモデルをbinに拡張していたりしています.

モデル概要は追ってみていきたいと思います.

まだ自然言語処理の基礎があやふやな部分があるので,モデルの詳細は後にみていきたいと思います.

## 3. 今後の方針

モデルの詳細を理解し追っていく.(11/20までにやる予定)  
自分のモデルを作成する.(11月末までにfirst sub)